In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re  # for regex
import seaborn as sns
import os
import unidecode
from autocorrect import Speller
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split, GroupKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, confusion_matrix, 
                             ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc, make_scorer)
from tqdm.notebook import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
# import tensorflow_hub as hub
# import bert_tokenization as tokenization
from transformers import *
from scipy.stats import spearmanr
from math import floor, ceil
from nltk.corpus import stopwords
np.set_printoptions(suppress=True)

print(tf.__version__)


C:\Users\grbha\anaconda3\Lib\site-packages\transformers\deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


C:\Users\grbha\anaconda3\Lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v4.40. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
C:\Users\grbha\anaconda3\Lib\site-packages\transformers\generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v4.40. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


2.16.1


In [2]:
from transformers import BertTokenizer, TFBertModel, BertModel, BertConfig


tf_checkpoint_path = 'bert'
MAX_SIZE = 200
BATCH_SIZE = 500

tokenizer = BertTokenizer.from_pretrained(tf_checkpoint_path)
model = BertModel.from_pretrained(tf_checkpoint_path)

loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file tokenizer.json
loading configuration file bert\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "architectures": [
    "ColBERT"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LAB

In [3]:
MAX_SENTENCE_LENGTH = 20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [4]:
humorousQ = pd.read_csv("Humorous.csv")
non_humorousQ = pd.read_csv("Non-humorous-unbiased.csv")

In [5]:
questions = pd.concat([humorousQ, non_humorousQ], ignore_index=True)
questions.rename(columns={'label': 'humor'}, inplace=True)
questions.drop(['product_description', 'image_url'], axis=1, inplace=True)
questions

,question,humor
0,Will the volca sample get me a girlfriend?,1
1,Can u communicate with spirits even on Saturday?,1
2,I won't get hunted right?,1
3,I have a few questions.. Can you get possessed...,1
4,Has anyone asked where the treasure is? What w...,1
...,...,...
19137,Serve na f800r 2013?,0
19138,Can it run mine sweeper abve 10 fps?,0
19139,What is the difference between the pro weight ...,0
19140,Can you provide me a phone number to the compa...,0


In [6]:
def case_convert():
    questions.question = [i.lower() for i in questions.question.values]

def remove_specials():
    questions.text =  [re.sub(r"[^a-zA-Z]"," ",text) for text in questions.question.values]

def remove_shorthands():
    CONTRACTION_MAP = {
    "u": "you",
    "abve": "above",
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }
    texts = []
    for text in questions.question.values:
        string = ""
        for word in text.split(" "):
            if word.strip() in list(CONTRACTION_MAP.keys()):
                string = string + " " + CONTRACTION_MAP[word]
            else:
                string = string + " " + word
        texts.append(string.strip())
    questions.question = texts

def remove_stopwords():
    texts = []
    stopwords_list = stopwords.words('english')
    for item in questions.question.values:
        string = ""
        for word in item.split(" "):
            if word.strip() in stopwords_list:
                continue
            else:
                string = string + " " + word
        texts.append(string)
                
def remove_links():
    texts = []
    for text in questions.question.values:
        remove_https = re.sub(r'http\S+', '', text)
        remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
        texts.append(remove_com)
    questions.question = texts

def remove_accents():
    questions.question = [unidecode.unidecode(text) for text in questions.question.values]

def normalize_spaces():
    questions.question = [re.sub(r"\s+"," ",text) for text in questions.question.values]

case_convert()
remove_links()
remove_shorthands()
remove_accents()
remove_specials()
remove_stopwords()
normalize_spaces()
print(questions)

C:\Users\grbha\AppData\Local\Temp\ipykernel_28556\4241353287.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  questions.text =  [re.sub(r"[^a-zA-Z]"," ",text) for text in questions.question.values]


                                                question  humor
0             will the volca sample get me a girlfriend?      1
1      can you communicate with spirits even on satur...      1
2                           i will not get hunted right?      1
3      i have a few questions.. can you get possessed...      1
4      has anyone asked where the treasure is? what w...      1
...                                                  ...    ...
19137                               serve na f800r 2013?      0
19138              can it run mine sweeper above 10 fps?      0
19139  what is the difference between the pro weight ...      0
19140  can you provide me a phone number to the compa...      0
19141                 is the blenders sponge latex free?      0

[19142 rows x 2 columns]


In [7]:
df_train, df_test = train_test_split(questions, test_size=0.2)

In [8]:
test_df_y = df_test.copy()
del df_test['humor']

df_sub = test_df_y.copy()

print(len(questions),len(df_train),len(df_test))
display(df_train.head())
display(df_test.head())

19142 15313 3829


,question,humor
13676,does it work on 220 volts or only 110?,0
3991,if i listen to these will i get arms like the ...,1
7107,is there a charity i can donate to so that i c...,1
15519,can this take a bb gun,0
18571,can i attach a real matte box to this rig with...,0


,question
1817,will this get me out of speeding tickets?
14009,would there be a shrek pillow pet in a size ap...
1655,does anyone notice the picture of the man driv...
11778,will the color of plastic stemless wine glasse...
4145,will this help me get a girlfriend? i have a j...


In [9]:
output_categories = list(df_train.columns[[1]])
input_categories = list(df_train.columns[[0]])

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['question']

output TARGET_COUNT:
	 1

output categories:
	 ['humor']


In [10]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\grbha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
def return_id(str1, str2, truncation_strategy, length):

    inputs = tokenizer.encode_plus(str1, str2,
        add_special_tokens=True,
        max_length=length,
        truncation_strategy=truncation_strategy)

    input_ids =  inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer):
    model_input = []
    for xx in range((MAX_SENTENCES*3)+3):
        model_input.append([])
    
    for _, row in tqdm(df[columns].iterrows()):
        i = 0
        
        # sent
        print(row)
        sentences = sent_tokenize(row.question)
        for xx in range(MAX_SENTENCES):
            s = sentences[xx] if xx<len(sentences) else ''
            ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
            model_input[i].append(ids_q)
            i+=1
            model_input[i].append(masks_q)
            i+=1
            model_input[i].append(segments_q)
            i+=1
        
        # full row
        ids_q, masks_q, segments_q = return_id(row.question, None, 'longest_first', MAX_LENGTH)
        model_input[i].append(ids_q)
        i+=1
        model_input[i].append(masks_q)
        i+=1
        model_input[i].append(segments_q)
        
    for xx in range((MAX_SENTENCES*3)+3):
        model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
        
    print(model_input[0].shape)
    return model_input

In [25]:
inputs      = compute_input_arrays(df_train, input_categories, tokenizer)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer)

0it [00:00, ?it/s]

question    does it work on 220 volts or only 110?
Name: 13676, dtype: object
question    if i listen to these will i get arms like the ...
Name: 3991, dtype: object
question    is there a charity i can donate to so that i c...
Name: 7107, dtype: object
question    can this take a bb gun
Name: 15519, dtype: object
question    can i attach a real matte box to this rig with...
Name: 18571, dtype: object
question    is it bendable?
Name: 16339, dtype: object
question    how big is it?
Name: 13209, dtype: object
question    my board started on fire while playing. is thi...
Name: 78, dtype: object
question    how can i use this to turn the frogs gay
Name: 5690, dtype: object
question    my neighbor egg'ed my house 3000+ times. if i ...
Name: 550, dtype: object
question    i come from the planet vulcan. will this help ...
Name: 8249, dtype: object
question    do you have any "make america native again" hats?
Name: 16177, dtype: object
question    where is this made?
Name: 13071, dtype: objec

question    can you use this indoors for a changing room? ...
Name: 14288, dtype: object
question    my lad who is 13 has asked for this and a diar...
Name: 9220, dtype: object
question    i am assuming that there is a museum dedicated...
Name: 1247, dtype: object
question    the site states free shipping but when i went ...
Name: 10236, dtype: object
question    does anyone know the company that makes this t...
Name: 12936, dtype: object
question    are these tools detachable or do they rest in ...
Name: 11176, dtype: object
question    did anyone buy this just to say they blew up t...
Name: 9428, dtype: object
question    is it ture 925 sterling silver?
Name: 16519, dtype: object
question    does the wallet come with the $100 in it?
Name: 7979, dtype: object
question    how do you do a cartwheel?
Name: 601, dtype: object
question    what type and how many batteries is needed for...
Name: 16364, dtype: object
question    do you still have this product with an updated...
Name: 15900, d

question    is this phone compatible with straight talk
Name: 2187, dtype: object
question    i love using darker bananas...they just taste ...
Name: 10757, dtype: object
question    did michael jackson use this?
Name: 6306, dtype: object
question    can i use a wireless keyboard with it?
Name: 13870, dtype: object
question    do you ship to puerto rico?
Name: 16590, dtype: object
question    is there a rounded version instead of straight...
Name: 13799, dtype: object
question    does it come with money
Name: 2924, dtype: object
question    what is the length and width of this product?
Name: 16850, dtype: object
question    is amazon prime required for use of the echo dot?
Name: 11203, dtype: object
question    does the bear come compressed or just shoved i...
Name: 11791, dtype: object
question    would humans be able to battle with them?
Name: 9277, dtype: object
question    how does this open to see the notes inside later
Name: 11631, dtype: object
question    to those who have one 

question    why is it so cheap?
Name: 12017, dtype: object
question    will this work in the zombie apocalypse?
Name: 9286, dtype: object
question    ingredients in precious cat litter
Name: 16512, dtype: object
question    can you actually grim reaper some of the town ...
Name: 4025, dtype: object
question    can anyone verify which version of the product...
Name: 12924, dtype: object
question    can this be shipped to ontario canada?
Name: 14082, dtype: object
question    does it come with a strobe light effect like t...
Name: 3697, dtype: object
question    what happens to all the baby rat nests also in...
Name: 4443, dtype: object
question    do he drink
Name: 7304, dtype: object
question    can i take other people's souls if i win?
Name: 6286, dtype: object
question    is this real
Name: 11532, dtype: object
question    works on comcast ?
Name: 16830, dtype: object
question    can this ornament summon the devil and bring a...
Name: 364, dtype: object
question    does it sow rise?


...

question    if i stick my tongue in a light socket while h...
Name: 353, dtype: object
question    does it record memories?
Name: 7236, dtype: object
question    are you for real? tell you what ... if i win t...
Name: 1465, dtype: object
question    humans?
Name: 5065, dtype: object
question    my phone calls to vladimir putin and other for...
Name: 6978, dtype: object
question    will this fit 2012 toyota corolla with spoiler?
Name: 18453, dtype: object
question    how long can it stay in water without getting ...
Name: 11892, dtype: object
question    the listing says no msg but there is monosodiu...
Name: 18752, dtype: object
question    how many hours does it play before you have to...
Name: 11688, dtype: object
question    is there a google play store remote app that w...
Name: 14185, dtype: object
question    i am buying this cable but i have had bad expe...
Name: 14041, dtype: object
question    can this printer make me a sandwich?
Name: 7572, dtype: object
question    can i cal

question    with an xbox one can i bring a gun into a chil...
Name: 5161, dtype: object
question    why is this offensive shit in my "recommendati...
Name: 11458, dtype: object
question    if i spill salsa on my baby's head can i just ...
Name: 9489, dtype: object
question    why would anyone even want this? get a smart w...
Name: 2608, dtype: object
question    can i get a refund or at least store credit if...
Name: 11366, dtype: object
question    would you really get the girl?
Name: 9396, dtype: object
question    will it make you a sandwich?
Name: 1436, dtype: object
question    does this accurately predict the future? i hav...
Name: 8197, dtype: object
question    what would happen if i put this in the microwave?
Name: 7002, dtype: object
question    you can sell it for colombia. i am very intere...
Name: 11675, dtype: object
question    where do the amish keep their buffalo?
Name: 6530, dtype: object
question    price? $122999.00 plus free shipping. really!!!
Name: 7075, dtype: o

question    if i buy it from amazon will it have the bonuses
Name: 16882, dtype: object
question    are half mask respirators as safe as full face...
Name: 18204, dtype: object
question    will this kill rabbits? sorry bugs bunny lover...
Name: 17796, dtype: object
question    is it a banana?
Name: 6722, dtype: object
question    is it lined with tinfoil?
Name: 10053, dtype: object
question    is is good to eat with cars 2?
Name: 6027, dtype: object
question    does it have the little heart inside the stuff...
Name: 16901, dtype: object
question    can you actually kill zombies with this?
Name: 6821, dtype: object
question    will this keep bible salesman from knocking on...
Name: 1862, dtype: object
question    is the item durable
Name: 15884, dtype: object
question    for $10 bucks will they come over and make me ...
Name: 9447, dtype: object
question    am i reading this correctly? does this product...
Name: 1687, dtype: object
question    who the hell spends 40k on a tv?
Name: 4537

question    is the ghost freindly
Name: 7086, dtype: object
question    will this help me catch flight attendants?
Name: 7118, dtype: object
question    can i pretend that my child is harambe and rai...
Name: 7517, dtype: object
question    what is the discount for prime member?
Name: 12211, dtype: object
question    want to order but i could not cancel first order
Name: 14103, dtype: object
question    is this latex the kind that latex senitive peo...
Name: 18803, dtype: object
question    how do fitted sheets fit over thick mattress a...
Name: 17752, dtype: object
question    i wanted to add this tv to my home insurance b...
Name: 9792, dtype: object
question    does the pie come with it?
Name: 5445, dtype: object
question    will the ugly looks of this eat my soul?
Name: 1164, dtype: object
question    does the girl come with the device?
Name: 2478, dtype: object
question    i have a display port only monitor and have on...
Name: 16416, dtype: object
question    is this price just f

question    does it need batteries
Name: 16106, dtype: object
question    can someone adopt my son for xmas
Name: 7325, dtype: object
question    i bought the orange one. is that the fastest c...
Name: 8193, dtype: object
question    i normally wear 2xl shirts should i order 3xl ...
Name: 10276, dtype: object
question    this is too much work is there a way to have t...
Name: 5888, dtype: object
question    are the alligators aggressive with cats? my wi...
Name: 8858, dtype: object
question    the color of the balls differ in the images wi...
Name: 14687, dtype: object
question    how many zombies can this kill? it looks made ...
Name: 3943, dtype: object
question    would we get a superstar free if we pre ord it
Name: 4434, dtype: object
question    does this help with training to battle frieza'...
Name: 7484, dtype: object
question    when are these going to ship? i purchased duri...
Name: 12516, dtype: object
question    will it give a horsepower boost too my 73 supe...
Name: 2211, 

In [27]:
print(len(inputs), len(inputs[0]), len(inputs[0][0]))

# check out input for 7th row
xx = 7
print(df_train.iloc[xx,0])
print(sent_tokenize(df_train.iloc[xx,0]))
inputs[0][xx], inputs[3][xx], inputs[6][xx], inputs[15][xx]

18 15313 20
my board started on fire while playing. is this normal?
['my board started on fire while playing.', 'is this normal?']


(array([ 101, 2026, 2604, 2318, 2006, 2543, 2096, 2652, 1012,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]),
 array([ 101, 2003, 2023, 3671, 1029,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]),
 array([101, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0]),
 array([ 101, 2026, 2604, 2318, 2006, 2543, 2096, 2652, 1012, 2003, 2023,
        3671, 1029,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, 

In [29]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

outputs = compute_output_arrays(df_train, output_categories)
outputs[:3]

array([[0],
       [1],
       [1]], dtype=int64)

In [32]:
# Evaluation Metrics
import sklearn
def print_evaluation_metrics(y_true, y_pred, label='', is_regression=True, label2=''):
    print('==================', label2)
    ### For regression
    if is_regression:
        print('mean_absolute_error',label,':', sklearn.metrics.mean_absolute_error(y_true, y_pred))
        print('mean_squared_error',label,':', sklearn.metrics.mean_squared_error(y_true, y_pred))
        print('r2 score',label,':', sklearn.metrics.r2_score(y_true, y_pred))
        #     print('max_error',label,':', sklearn.metrics.max_error(y_true, y_pred))
        return sklearn.metrics.mean_squared_error(y_true, y_pred)
    else:
        ### FOR Classification
#         print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
#         print('average_precision_score',label,':', sklearn.metrics.average_precision_score(y_true, y_pred))
#         print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
#         print('accuracy_score',label,':', sklearn.metrics.accuracy_score(y_true, y_pred))
        print('f1_score',label,':', sklearn.metrics.f1_score(y_true, y_pred))
        
        matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
        print(matrix)
        TP,TN,FP,FN = matrix[1][1],matrix[0][0],matrix[0][1],matrix[1][0]
        Accuracy = (TP+TN)/(TP+FP+FN+TN)
        Precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1 = 2*(Recall * Precision) / (Recall + Precision)
        print('Acc', Accuracy, 'Prec', Precision, 'Rec', Recall, 'F1',F1)
        return sklearn.metrics.accuracy_score(y_true, y_pred)


In [33]:
valid_inputs = inputs
valid_outputs = outputs

In [34]:
preds = model.predict(valid_inputs)


AttributeError: 'BertModel' object has no attribute 'predict'